In [18]:
import pandas as pd
trans_data = '/home/voyager/data/lz_graph/2015-7data.csv'
trans_data = pd.read_csv(trans_data,header = 0 )
trans_data.head(2)                                         #ACCTN0 兰州银行客户账号，ACCTNO1 对手账号   #C 进账 ，D 支出  #customtype 1私人账号

/home/voyager/anaconda3/envs/graph_pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,TRANSOURCE,TIME,NAME,ACCTNO,ACCTNO1,TRANAMT,CDFLAG,CUSTOMTYPE
0,88,2015-07-01-00.09.48.687448,NaN,101661000111853,NaN,0.0,C,$null$
1,61,2015-07-01-00.00.34.024563,郭永刚,78652380011,6214968250550032066,40000.0,D,1


## 1.CLEAN

In [19]:
trans_data["CUSTOMTYPE"].unique()

array(['$null$', '1', '2', '3'], dtype=object)

In [20]:
import dgl
import numpy as np
import math
nodes_list = trans_data["ACCTNO"].to_list()
nodes_list.extend(trans_data["ACCTNO1"])
nodes_list = list(set(nodes_list))
nodes_list = [x for x in nodes_list  if (str(x)) !='nan' ]
nodes_data = pd.DataFrame(nodes_list,columns=['ACCTNO'])   #有type的账号
print(nodes_data.shape)

list1 = trans_data.filter(items=["ACCTNO","NAME","CUSTOMTYPE"]).drop_duplicates()
list1 = list1.loc[list1["CUSTOMTYPE"] != '$null$', :]
list1 = list1.loc[list1["ACCTNO"].isin(nodes_list),:]
nodes_data = pd.merge(nodes_data,list1,how ='left',on="ACCTNO").reset_index()
nodes_data["CUSTOMTYPE"] = nodes_data["CUSTOMTYPE"].fillna("0")   #4 for missing value

print(nodes_data.shape)
print(nodes_data["CUSTOMTYPE"].unique())

missing = []
missing.insert(0, {"ACCTNO":0,"CUSTOMTYPE":0,"NAME":"MISSING"})
nodes_data = pd.concat([pd.DataFrame(missing),nodes_data],ignore_index=True,axis =0)

nodes_data.insert(loc = 0 ,column = 'id', value = nodes_data.index)
nodes_data.drop(columns = ["index"] , inplace = True)

nodes_data.to_csv('../lz-graph/data/nodes.csv')
nodes_data.head(5)

(1630996, 1)
(1631198, 4)
['0' '1' '2' '3']


/home/voyager/anaconda3/envs/graph_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,id,ACCTNO,CUSTOMTYPE,NAME
0,0,0,0,MISSING
1,1,6214664260258704,0,NaN
2,2,101971000295428,1,岳姗姗
3,3,101541000249350,1,陶婧
4,4,101221000019976,1,周燕云


In [21]:
max(nodes_data.index)

1631198

In [22]:
links = pd.merge(trans_data,nodes_data.filter(items = ["ACCTNO","id"]),how='left',on ='ACCTNO')
links = links.rename(columns = {'id': 'source'})
test1 = nodes_data.filter(items = ["ACCTNO","id"])
test1.columns = ["ACCTNO1","target"]
                         
links = pd.merge(links,test1,how='left',left_on ='ACCTNO1',right_on = 'ACCTNO1')
#links = links.rename(columns = {'id': 'target'})
links["source1"] = links["source"]
links.loc[links["CDFLAG"]=="D","source"] = links.loc[links["CDFLAG"]=="D","target"]
links.loc[links["CDFLAG"]=="D","target"] = links.loc[links["CDFLAG"]=="D","source1"]
links.drop(columns=["source1"] ,inplace = True)
links = links.filter(items = ["source","target","TIME","TRANSOURCE","ACCTNO","ACCTNO1","TRANAMT","CDFLAG"])
links["source"] = links["source"].fillna(0).astype("int")
links["target"] = links["target"].fillna(0).astype("int")
links.to_csv('../lz-graph/data/links.csv')
links.head(5)

,source,target,TIME,TRANSOURCE,ACCTNO,ACCTNO1,TRANAMT,CDFLAG
0,327880,0,2015-07-01-00.09.48.687448,88,101661000111853,NaN,0.0,C
1,338506,316022,2015-07-01-00.00.34.024563,61,78652380011,6214968250550032066,40000.0,D
2,687244,1471289,2015-07-01-00.00.31.895795,30,102561000017549,101870122410308000002,2000.0,D
3,534095,167735,2015-07-01-00.00.20.384808,73,101171000141298,101870122410308001000,1.0,D
4,534095,625375,2015-07-01-00.00.11.019973,73,101671000073599,101870122410308001000,500.0,D


In [23]:
type(links["TIME"])

pandas.core.series.Series

In [24]:
import time
links["timestamp"] = (links["TIME"].str[0:19])
#links["timestamp"] = pd.to_datetime(links["timestamp"],format = '%Y-%m-%d-%H.%M.%S' )
links["timestamp"]  = links["timestamp"].apply(lambda x : time.mktime(time.strptime(x,'%Y-%m-%d-%H.%M.%S')   )) 
links["ACCTNO"] = links["ACCTNO"].astype(str)  #
links["ACCTNO1"] = links["ACCTNO1"].astype(str) 
links.head(2)

,source,target,TIME,TRANSOURCE,ACCTNO,ACCTNO1,TRANAMT,CDFLAG,timestamp
0,327880,0,2015-07-01-00.09.48.687448,88,101661000111853,nan,0.0,C,1.435681e+09
1,338506,316022,2015-07-01-00.00.34.024563,61,78652380011,6214968250550032066,40000.0,D,1.435680e+09


## convert edges features to node features

In [ ]:
stat1=links.groupby(['source','target'])["TRANAMT"].agg(["count","sum","mean"]).reset_index()

# 2.stat

In [9]:
stat1=links.groupby(['source','target'])["TRANAMT"].agg(["count","sum"]).reset_index()
stat1 = pd.DataFrame(stat1)
stat1.to_csv('../lz-graph/data/stat_links.csv')
stat1.head(3)

,source,target,count,sum
0,0,2,2,29800.0
1,0,4,4,15000.0
2,0,7,1,5.0


In [ ]:
# 筛选 2015-07-16 这天的数据


## 3.GRAPH

In [10]:
cleans = links.loc[links["TRANAMT"]>1000,:]
import torch as th
#nodes_id = th.tensor(nodes_data["id"].astype("int").values)
def build_graph(nodes,links):
    g = dgl.DGLGraph()
    # add 34 nodes into the graph; nodes are labeled from 0~33
    g.add_nodes(len(nodes))
    edge_list = [tuple(x) for x in links[["source","target"]].values]
    src, dst = tuple(zip(*edge_list))
    print(type(src))
    g.add_edges(src,dst)
    #g.edata["y"] = links.as_matrix()
    return g

In [11]:
G = build_graph(nodes = nodes_data,links = cleans)
print('We have %d nodes.' % G.number_of_nodes())
print('We have %d edges.' % G.number_of_edges())
print(type(G))

<class 'tuple'>
We have 1631199 nodes.
We have 2564961 edges.
<class 'dgl.graph.DGLGraph'>


## 4.visalisation 
(数据太多，无法显示)

In [ ]:
import networkx as nx
nx_G = G.to_networkx().to_directed()
# Kamada-Kawaii layout usually looks pretty for arbitrary graphs
pos = nx.kamada_kawai_layout(nx_G)
nx.draw(nx_G, pos, with_labels=True, node_color=[[.7, .7, .7]])

## 5.GCN

In [12]:
import torch.nn as nn
import torch.nn.functional as F

# Define the message & reduce function
# NOTE: we ignore the GCN's normalization constant c_ij for this tutorial.
def gcn_message(edges):
    # The argument is a batch of edges.
    # This computes a (batch of) message called 'msg' using the source node's feature 'h'.
    return {'msg' : edges.src['h']}

def gcn_reduce(nodes):
    # The argument is a batch of nodes.
    # This computes the new 'h' features by summing received 'msg' in each node's mailbox.
    return {'h' : torch.sum(nodes.mailbox['msg'], dim=1)}

# Define the GCNLayer module
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, inputs):
        # g is the graph and the inputs is the input node features
        # first set the node features                                                  #g:仅仅是图结构，没有节点属性和边属性
        g.ndata['h'] = inputs                                                          #don't set edges features
        # trigger message passing on all edges
        g.send(g.edges(), gcn_message)
        # trigger aggregation at all nodes
        g.recv(g.nodes(), gcn_reduce)                                                  #按每个点做了聚合===》新的features
        # get the result node features
        h = g.ndata.pop('h')
        # perform linear transformation
        return self.linear(h)

In [13]:
# Define a 2-layer GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_feats, hidden_size)
        self.gcn2 = GCNLayer(hidden_size, num_classes)

    def forward(self, g, inputs):
        h = self.gcn1(g, inputs)
        h = torch.relu(h)
        h = self.gcn2(g, h)
        return h
# The first layer transforms input features of size of 34 to a hidden size of 5.
# The second layer transforms the hidden layer and produces output features of
# size 2, corresponding to the two groups of the karate club.
net = GCN(4,3, 2)

In [14]:
import torch

customtype = pd.get_dummies(nodes_data["CUSTOMTYPE"].astype(str)).as_matrix()   #convert pandas to pytorch tensor
inputs = th.tensor(customtype,dtype = torch.float)   #哑变量  ,,dtype=torch.long  #https://wsonh.com/article/6.html   #nodes的features
id0 = nodes_data.loc[nodes_data["ACCTNO"]=="102561000017549","id"].values[0]    #来源于数据源，任意取
id1 = nodes_data.loc[nodes_data["ACCTNO"]=="101091000178241","id"].values[0]    #来源于异常报告，挑选的几个典型的异常点  ，#2015/7/16
id2 = nodes_data.loc[nodes_data["ACCTNO"]=="102192000158598","id"].values[0]                                           #2015/7/16
#id3 = nodes_data.loc[nodes_data["ACCTNO"]=="102191000178651","id"].values[0]  #暂时，找不到这个异常点

labeled_nodes = th.tensor([id0 ,id1, id2 ])  # only the instructor and the president nodes are labeled
labels = th.tensor([0,1,1])  # their labels are different

/home/voyager/anaconda3/envs/graph_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
print(inputs.type())  #torch.unit8

torch.FloatTensor


In [16]:
%%time
import torch
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)  #创建优化器
all_logits = []
# move to gpu
device = torch.device('cuda')

net = net.to(device)
inputs = inputs.to(device)
labels = labels.to(device)

for epoch in range(2):
    logits = net(G, inputs)
    # we save the logits for visualization later
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)
    #print(logp)
    # we only compute loss for labeled nodes
    loss = F.nll_loss(logp[labeled_nodes], labels)   #mask 可以看重是标记层

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))

/home/voyager/anaconda3/envs/graph_pytorch/lib/python3.6/site-packages/dgl/base.py:18: UserWarning: Initializer is not set. Use zero initializer instead. To suppress this warning, use `set_initializer` to explicitly specify which initializer to use.
  warnings.warn(msg)


tensor([[-2.6874e+05,  0.0000e+00],
        [-3.6490e-01, -1.1850e+00],
        [-3.6490e-01, -1.1850e+00],
        ...,
        [-3.7825e-01, -1.1554e+00],
        [-4.0616e-01, -1.0972e+00],
        [-3.6490e-01, -1.1850e+00]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
Epoch 0 | Loss: 0.3127
tensor([[-3.1792e+05,  0.0000e+00],
        [-3.7106e-01, -1.1712e+00],
        [-3.7106e-01, -1.1712e+00],
        ...,
        [-3.8682e-01, -1.1370e+00],
        [-4.1997e-01, -1.0702e+00],
        [-3.7106e-01, -1.1712e+00]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
Epoch 1 | Loss: 0.2884
CPU times: user 19min 22s, sys: 3.42 s, total: 19min 25s
Wall time: 19min 23s
